# Financial Analysis Toolkit - Retirement Planning Walkthrough

This notebook demonstrates the core capabilities of the Financial Analysis Toolkit for retirement planning analysis. We'll explore both the Python API and CLI interfaces to show how you can perform Monte Carlo simulations for retirement portfolio sustainability.

## Overview

The toolkit provides:
- **Multiple withdrawal strategies**: 4% rule, Guyton-Klinger guardrails, constant percentage, and more
- **Monte Carlo simulations**: Run thousands of scenarios to test portfolio resilience
- **Flexible configuration**: Use YAML files, CLI arguments, or Python API
- **Rich visualizations**: Equity curves, success rates, and detailed reporting

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

# Add the src directory to the Python path
sys.path.insert(0, str(Path('../src').resolve()))

# Import the Financial Analysis Toolkit
import capstone_finance as cf
from capstone_finance.core.ledger import CashFlowLedger
from capstone_finance.core.market import MarketSimulator
from capstone_finance.core.models import PortfolioParams
from capstone_finance.reporting.summary import create_summary_statistics
from capstone_finance.strategies.constant_percentage import ConstantPercentageStrategy
from capstone_finance.strategies.four_percent_rule import FourPercentRule
from capstone_finance.strategies.guyton_klinger import GuytonKlingerStrategy

# Set up matplotlib for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print(f"Financial Analysis Toolkit version: {cf.__version__}")
print("Environment setup complete!")

## Part 1: Basic Retirement Simulation with Python API

Let's start with a basic retirement simulation using the classic 4% withdrawal rule. We'll set up a $1 million portfolio with 60% equity allocation and simulate 30 years of retirement.

In [ ]:
# Create portfolio parameters
portfolio_params = PortfolioParams(
    init_balance=1_000_000.0,
    equity_pct=0.6,
    fees_bps=50,  # 0.5% annual fees
    seed=42
)

# Initialize the 4% rule strategy
strategy = FourPercentRule()

# Create market simulator
market_sim = MarketSimulator(mode="lognormal", seed=42)

# Create the cash flow ledger for running simulations
ledger = CashFlowLedger()

# Run the simulation
print("Running 4% Rule simulation...")
results_df = ledger.run_simulation(
    params=portfolio_params,
    strategy=strategy,
    market_sim=market_sim,
    years=30,
    paths=1000
)

print(f"Simulation complete! Generated {len(results_df)} data points.")
print("\nFirst few rows of results:")
print(results_df.head())

### Analyzing the Results

Let's analyze the simulation results to understand the portfolio's performance:

In [ ]:
# Calculate summary statistics
summary_stats = create_summary_statistics(results_df)

print("=== 4% Rule Simulation Results ===")
print(f"Success Rate: {summary_stats['success_rate']:.1%}")
print(f"Median Final Balance: ${summary_stats['median_final_balance']:,.0f}")
print(f"10th Percentile Final Balance: ${summary_stats['p10_final_balance']:,.0f}")
print(f"90th Percentile Final Balance: ${summary_stats['p90_final_balance']:,.0f}")
print(f"Average Annual Withdrawal: ${summary_stats['avg_withdrawal']:,.0f}")
print(f"Total Simulation Paths: {summary_stats['total_paths']}")

# Calculate failure rate and years to failure
final_year_data = results_df[results_df['year'] == 30]
failures = final_year_data[final_year_data['balance'] <= 0]
failure_rate = len(failures) / len(final_year_data)

print("\n=== Risk Analysis ===")
print(f"Failure Rate: {failure_rate:.1%}")
print(f"Number of Failed Paths: {len(failures)} out of {len(final_year_data)}")

### Visualizing the Results

Let's create visualizations to better understand the portfolio trajectories:

In [ ]:
# Create equity curve plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Portfolio balance trajectories
# Show percentile bands
yearly_stats = results_df.groupby('year')['balance'].agg([
    lambda x: x.quantile(0.1),   # 10th percentile
    lambda x: x.quantile(0.25),  # 25th percentile
    'median',                    # 50th percentile
    lambda x: x.quantile(0.75),  # 75th percentile
    lambda x: x.quantile(0.9)    # 90th percentile
])
yearly_stats.columns = ['p10', 'p25', 'median', 'p75', 'p90']

years = yearly_stats.index
ax1.fill_between(years, yearly_stats['p10'], yearly_stats['p90'], alpha=0.2, color='blue', label='10th-90th percentile')
ax1.fill_between(years, yearly_stats['p25'], yearly_stats['p75'], alpha=0.3, color='blue', label='25th-75th percentile')
ax1.plot(years, yearly_stats['median'], color='blue', linewidth=2, label='Median')
ax1.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Portfolio Depletion')

ax1.set_title('4% Rule: Portfolio Balance Over Time', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year in Retirement')
ax1.set_ylabel('Portfolio Balance ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Plot 2: Success rate over time
success_by_year = results_df.groupby('year').apply(
    lambda x: (x['balance'] > 0).mean()
)

ax2.plot(success_by_year.index, success_by_year.values * 100, color='green', linewidth=2, marker='o', markersize=4)
ax2.set_title('Portfolio Success Rate Over Time', fontsize=14, fontweight='bold')
ax2.set_xlabel('Year in Retirement')
ax2.set_ylabel('Success Rate (%)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 105)

plt.tight_layout()
plt.show()

print(f"Final success rate after 30 years: {success_by_year.iloc[-1]:.1%}")

## Part 2: Comparing Multiple Withdrawal Strategies

Now let's compare different withdrawal strategies to see how they perform under the same market conditions:

In [ ]:
# Define strategies to compare
strategies = {
    "4% Rule": FourPercentRule(),
    "3% Constant": ConstantPercentageStrategy(percentage=0.03),
    "Guyton-Klinger": GuytonKlingerStrategy(
        initial_rate=0.045,
        guard_pct=0.20,
        raise_pct=0.10,
        cut_pct=0.10
    )
}

# Run simulations for each strategy
strategy_results = {}
strategy_summaries = {}

for name, strategy in strategies.items():
    print(f"Running simulation for {name}...")

    # Use the same market simulator for fair comparison
    market_sim = MarketSimulator(mode="lognormal", seed=42)

    results = ledger.run_simulation(
        params=portfolio_params,
        strategy=strategy,
        market_sim=market_sim,
        years=30,
        paths=1000
    )

    strategy_results[name] = results
    strategy_summaries[name] = create_summary_statistics(results)

print("\nAll strategy simulations complete!")

### Strategy Comparison Analysis

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Strategy': list(strategy_summaries.keys()),
    'Success Rate': [f"{s['success_rate']:.1%}" for s in strategy_summaries.values()],
    'Median Final Balance': [f"${s['median_final_balance']:,.0f}" for s in strategy_summaries.values()],
    'Avg Annual Withdrawal': [f"${s['avg_withdrawal']:,.0f}" for s in strategy_summaries.values()],
    'P10 Final Balance': [f"${s['p10_final_balance']:,.0f}" for s in strategy_summaries.values()]
})

print("=== Strategy Comparison ===")
print(comparison_df.to_string(index=False))

# Create side-by-side comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Strategy Comparison: Portfolio Trajectories', fontsize=16, fontweight='bold')

colors = ['blue', 'green', 'orange']

for i, (name, results) in enumerate(strategy_results.items()):
    ax = axes[i//2, i%2]

    # Calculate percentiles
    yearly_stats = results.groupby('year')['balance'].agg([
        lambda x: x.quantile(0.1),
        'median',
        lambda x: x.quantile(0.9)
    ])
    yearly_stats.columns = ['p10', 'median', 'p90']

    years = yearly_stats.index
    color = colors[i]

    ax.fill_between(years, yearly_stats['p10'], yearly_stats['p90'], alpha=0.2, color=color)
    ax.plot(years, yearly_stats['median'], color=color, linewidth=2, label='Median')
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.7)

    ax.set_title(f'{name}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Year in Retirement')
    ax.set_ylabel('Portfolio Balance ($)')
    ax.grid(True, alpha=0.3)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Remove the empty subplot
axes[1, 1].remove()

plt.tight_layout()
plt.show()

## Part 3: Using Configuration Files (CLI Integration)

The toolkit also supports configuration files for more complex scenarios. Let's demonstrate how to use the existing configuration files:

In [ ]:
# Let's examine the configuration files
config_dir = Path('../examples')
config_files = list(config_dir.glob('*.yml'))

print("Available configuration files:")
for config_file in config_files:
    print(f"  - {config_file.name}")

# Read and display the basic config
print("\n=== retirement_basic.yml ===")
with open(config_dir / 'retirement_basic.yml') as f:
    print(f.read())

### CLI Commands Demonstration

Here are the equivalent CLI commands for our Python API examples:

In [ ]:
# Show CLI command examples
print("=== CLI Command Examples ===")
print("\n1. Basic 4% Rule simulation:")
print("   finance-cli retire --strategy four_percent_rule --years 30 --paths 1000 --seed 42")

print("\n2. Using configuration file:")
print("   finance-cli retire --config examples/retirement_basic.yml")

print("\n3. Overriding config with CLI parameters:")
print("   finance-cli retire --config examples/retirement_basic.yml --years 40 --strategy guyton_klinger")

print("\n4. Guyton-Klinger with custom parameters:")
print("   finance-cli retire --config examples/guyton_klinger_example.yml --verbose")

print("\n5. Constant percentage strategy:")
print("   finance-cli retire --strategy constant_pct --percent 0.03 --years 25 --paths 500")

print("\n6. Export results to CSV:")
print("   finance-cli retire --strategy four_percent_rule --output my_results.csv")

print("\n7. List available strategies:")
print("   finance-cli list-strategies")

## Part 4: Advanced Analysis - Monte Carlo Sensitivity

Let's explore how the number of Monte Carlo paths affects the stability of our results:

In [ ]:
# Test different path counts
path_counts = [100, 500, 1000, 2000, 5000]
convergence_results = []

strategy = FourPercentRule()

for paths in path_counts:
    print(f"Running simulation with {paths} paths...")

    # Use different seeds to avoid exact replication
    market_sim = MarketSimulator(mode="lognormal", seed=42)

    results = ledger.run_simulation(
        params=portfolio_params,
        strategy=strategy,
        market_sim=market_sim,
        years=30,
        paths=paths
    )

    summary = create_summary_statistics(results)
    convergence_results.append({
        'paths': paths,
        'success_rate': summary['success_rate'],
        'median_final': summary['median_final_balance'],
        'avg_withdrawal': summary['avg_withdrawal']
    })

convergence_df = pd.DataFrame(convergence_results)
print("\n=== Monte Carlo Convergence Analysis ===")
print(convergence_df.to_string(index=False))

# Plot convergence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.plot(convergence_df['paths'], convergence_df['success_rate'] * 100, 'bo-', linewidth=2, markersize=8)
ax1.set_title('Success Rate Convergence', fontsize=12, fontweight='bold')
ax1.set_xlabel('Number of Monte Carlo Paths')
ax1.set_ylabel('Success Rate (%)')
ax1.grid(True, alpha=0.3)

ax2.plot(convergence_df['paths'], convergence_df['median_final']/1e6, 'ro-', linewidth=2, markersize=8)
ax2.set_title('Median Final Balance Convergence', fontsize=12, fontweight='bold')
ax2.set_xlabel('Number of Monte Carlo Paths')
ax2.set_ylabel('Median Final Balance ($M)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRecommendation: Use at least 1,000 paths for stable results.")
print(f"The difference between 1,000 and 5,000 paths is {abs(convergence_df.iloc[-1]['success_rate'] - convergence_df.iloc[2]['success_rate']):.1%}")

## Part 5: Creating Custom Scenarios

Let's create a custom scenario to demonstrate the flexibility of the toolkit:

In [ ]:
# Custom scenario: Early retirement with lower initial balance
early_retirement_params = PortfolioParams(
    init_balance=750_000.0,  # Lower starting balance
    equity_pct=0.8,          # Higher equity allocation (more aggressive)
    fees_bps=25,             # Lower fees (index fund portfolio)
    seed=123
)

# Compare conservative vs aggressive withdrawal strategies
conservative_strategy = ConstantPercentageStrategy(percentage=0.025)  # 2.5%
aggressive_strategy = FourPercentRule()

scenarios = {
    "Conservative (2.5%)": conservative_strategy,
    "Aggressive (4% Rule)": aggressive_strategy
}

print("=== Early Retirement Scenario ===")
print(f"Starting balance: ${early_retirement_params.init_balance:,.0f}")
print(f"Equity allocation: {early_retirement_params.equity_pct:.0%}")
print(f"Annual fees: {early_retirement_params.fees_bps/100:.2f}%")
print()

scenario_results = {}

for name, strategy in scenarios.items():
    print(f"Running {name} scenario...")

    market_sim = MarketSimulator(mode="lognormal", seed=123)

    results = ledger.run_simulation(
        params=early_retirement_params,
        strategy=strategy,
        market_sim=market_sim,
        years=40,  # Longer retirement period
        paths=1000
    )

    scenario_results[name] = results
    summary = create_summary_statistics(results)

    print(f"  Success rate: {summary['success_rate']:.1%}")
    print(f"  Avg annual withdrawal: ${summary['avg_withdrawal']:,.0f}")
    print(f"  Median final balance: ${summary['median_final_balance']:,.0f}")
    print()

# Visualize the comparison
fig, ax = plt.subplots(figsize=(12, 8))

colors = ['blue', 'red']
for i, (name, results) in enumerate(scenario_results.items()):
    yearly_stats = results.groupby('year')['balance'].median()
    ax.plot(yearly_stats.index, yearly_stats.values/1e6, color=colors[i], linewidth=2, label=name)

ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.set_title('Early Retirement Scenario: Conservative vs Aggressive Withdrawal', fontsize=14, fontweight='bold')
ax.set_xlabel('Year in Retirement')
ax.set_ylabel('Median Portfolio Balance ($M)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary and Key Takeaways

This walkthrough demonstrated the key capabilities of the Financial Analysis Toolkit:

### What We Learned:
1. **Multiple Interfaces**: The toolkit provides both Python API and CLI interfaces for maximum flexibility
2. **Strategy Comparison**: Different withdrawal strategies have varying risk/return profiles
3. **Monte Carlo Power**: Simulating thousands of scenarios provides robust insights into portfolio sustainability
4. **Configuration Flexibility**: YAML files make it easy to define and share complex scenarios
5. **Visualization**: Rich plotting capabilities help interpret results visually

### Best Practices:
- Use at least 1,000 Monte Carlo paths for stable results
- Compare multiple withdrawal strategies for your specific situation
- Consider longer time horizons for early retirement scenarios
- Pay attention to equity allocation and fees - they significantly impact outcomes
- Always include a margin of safety in your withdrawal rate

### Next Steps:
- Explore the CLI interface with `finance-cli --help`
- Try different market simulation modes (lognormal vs bootstrap)
- Create custom withdrawal strategies using the plugin architecture
- Integrate with your own data sources and analysis workflows

### Important Disclaimer:
**This toolkit is for educational purposes only and does not constitute financial advice.** Always consult with qualified financial professionals before making investment decisions.

In [ ]:
# Generate a final summary report
print("=== Final Summary Report ===")
print(f"Financial Analysis Toolkit version: {cf.__version__}")
print(f"Total simulations run: {len(strategy_results) + len(scenario_results)}")
print(f"Total Monte Carlo paths: {sum(len(df) for df in list(strategy_results.values()) + list(scenario_results.values()))}")
print("\nStrategies tested:")
for strategy_name in list(strategy_results.keys()) + list(scenario_results.keys()):
    print(f"  - {strategy_name}")

print("\nNotebook execution complete! 🎉")